In [24]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\Orel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Collecting xgboost
  Downloading xgboost-1.7.3-py3-none-win_amd64.whl (89.1 MB)
     --------------------------------------- 89.1/89.1 MB 12.8 MB/s eta 0:00:00
Requirement already satisfied: numpy in c:\users\mosac\anaconda3\lib\site-packages (from xgboost) (1.21.5)
Requirement already satisfied: scipy in c:\users\mosac\anaconda3\lib\site-packages (from xgboost) (1.9.1)
Installing collected packages: xgboost
Successfully installed xgboost-1.7.3
Note: you may need to restart the kernel to use updated packages.

In [25]:
%pip install opencv-python


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\Orel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [26]:
%pip install mediapipe 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\Orel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [27]:
import mediapipe as mp
import cv2
import numpy as np
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from  xgboost import XGBClassifier
import xgboost
from sklearn.metrics import r2_score
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [28]:
def calculate_angles(a,b,c):
    a = np.array(a) #hip
    b = np.array(b) #knee
    c = np.array(c) #heel
    #Calculates the angle formed by the three dots
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)#Convert to degrees
    #An angle cannot be more than 180
    if angle > 180.0:
        angle = 360- angle

    return angle

In [ ]:
#3 angale
data = {
    'albow': [],    
    'knee': [],    
    'shoulder': [],
              
    'label': [] #The value of the character in terms of the X-axis
}



In [ ]:
#run over all the images in images folder
directory = r'C:\Users\Orel\Documents\GitHub\FitnessAI\images'
label = ''
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
    
        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            #run over the images inside the folders
            #takes every image and put it inside a frame,f=image
            frame = cv2.imread(f, cv2.IMREAD_ANYCOLOR)#read the image to frame of cv2

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)#paint the pose
            image.flags.writeable = False
      
            results = pose.process(image)#the picther with the scale and dots

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)#change the color format

            #Extract  the dots from the image,every dots contain x,y,z,visabillity and saved in arr
            try:
                landmarks = results.pose_landmarks.landmark
            except:
                pass

            #drew the dots
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), #different color-blue
                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) #different color-pink
                            )
         #binary classification between d,s,b
         #ask in the folder give label by the name written in the image
        if 'deadlift' in f:
            label = 'deadlift'
        elif 'squat' in f:
            label = 'squat'
        elif 'bench' in f:
            label = 'bench'
        else:
            continue
        
        
        #calculate desired angels.
        #add those angels values to the data array.
        #33 landmark,11 is the first landmark that she isnt face
        #Can Be Deleted
        for i in range(11,len(mp_pose.PoseLandmark)-10):#run over all the lendmark of one imag
            data["albow"].append(landmarks[i].x)#add the landmark
            data["knee"].append(landmarks[i].y)
            data["shoulder"].append(landmarks[i].z)
            data["label"].append(label)